In [23]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available)

1.10.1
11.3
8200
NVIDIA GeForce RTX 3070 Laptop GPU
<function is_available at 0x00000180BAF395E8>


In [ ]:
images = torch.randn(2, 3, 4).cuda()
print('images: ',images)
knn(images, 2)


In [43]:
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)  # torch.Size([16, 1024, 1024])
    # print('inner: ',inner.shape)
    xx = torch.sum(x**2, dim=1, keepdim=True)  # torch.Size([16, 1, 1024])
    # print('xx: ',xx)
    # print('xx: ',xx.shape)
    # print('-xx - inner: ',-xx - inner)
    # print('-xx - inner: ',(-xx - inner).shape)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)  # torch.Size([16, 1024, 1024])
    # -(x + x**(-1))**2
    # print('pairwise_distance: ',pairwise_distance)
    # print('pairwise_distance: ',pairwise_distance.shape)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    # print('idx: ', idx)

    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0) # 16
    num_points = x.size(2) # 1024
    device = torch.device('cuda')
    x = x.view(batch_size, -1, num_points) # torch.Size([16, 3, 1024])
    print('ggf_x ',x.shape)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k) torch.Size([16, 1024, 20])
        # print(idx.shape)
    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points # torch.Size([16, 1, 1])
    # print(idx_base.shape)

    idx = idx + idx_base

    idx = idx.view(-1)  # torch.Size([327680])
    print(idx.shape)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()  # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) 
                                        # batch_size * num_points * k + range(0, batch_size*num_points)
                                        
    feature = x.view(batch_size*num_points, -1)[idx, :]

    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    # x  torch.Size([16, 1024, 20, 3])
    # feature  torch.Size([16, 1024, 20])
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
    # torch.Size([16, 6, 1024, 20])
    return feature

In [44]:
model_ = DGCNN().cuda()
pre = model_(images)

x  torch.Size([16, 3, 1024])
ggf_x  torch.Size([16, 3, 1024])
inner:  torch.Size([16, 1024, 1024])
xx:  torch.Size([16, 1, 1024])
-xx - inner:  torch.Size([16, 1024, 1024])
pairwise_distance:  torch.Size([16, 1024, 1024])
idx:  tensor([[[   0,  311,   67,  ...,  890,  642,  185],
         [   1,  346,  554,  ...,  331,  551,  639],
         [   2,  104,  902,  ...,  980,  594,  461],
         ...,
         [1021,   39,  718,  ...,  420, 1019,  413],
         [1022,    6,  146,  ...,  736,  615,   34],
         [1023,  414,  566,  ...,  988,  825,  580]],

        [[   0,   97,   42,  ...,  593,   29,  831],
         [   1,  945,  946,  ...,  779,    6,  605],
         [   2,  756,   26,  ...,  634,  205,  888],
         ...,
         [1021,  651,  499,  ...,  199,  807,  598],
         [1022,  606,  486,  ...,  714,   32,  603],
         [1023,  492,  445,  ...,  824,  468,  458]],

        [[   0,  757,  955,  ...,  915, 1020,  505],
         [   1,  885,  789,  ...,  536,  157,  669]

In [25]:
class DGCNN(nn.Module):
    def __init__(self, output_channels=40):
        super(DGCNN, self).__init__()
        # self.args = args
        self.k = 20
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(1024)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, 1024, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(1024*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=0.5)
        self.linear3 = nn.Linear(256, output_channels)

    def forward(self, x):
        batch_size = x.size(0)
        print('x ',x.shape)
        # torch.Size([16, 3, 1024])
        x = get_graph_feature(x, k=self.k)  # torch.Size([16, 6, 1024, 20])
        print('x ',x.shape)
        x = self.conv1(x)  # torch.Size([16, 64, 1024, 20])
        # print('x ',x.shape)
        x1 = x.max(dim=-1, keepdim=False)[0]  # torch.Size([16, 64, 1024])
        # print('x1 ',x1.shape)

        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]  # torch.Size([16, 64, 1024])
        # print('x2 ',x2.shape)

        x = get_graph_feature(x2, k=self.k)  
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]  # torch.Size([16, 128, 1024])
        # print('x3 ',x3.shape)

        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]  # torch.Size([16, 256, 1024])
        # print('x4 ',x4.shape)

        x = torch.cat((x1, x2, x3, x4), dim=1)

        x = self.conv5(x)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)

        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
        x = self.dp2(x)
        x = self.linear3(x)
        return x

In [41]:
images = torch.randn(16, 3, 1024).cuda()

In [9]:
# from model import DGCNN
model_ = DGCNN().cuda()
pre = model_(images)
# pre

x  torch.Size([16, 3, 1024])
x  torch.Size([16, 6, 1024, 20])


In [22]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'